Through these experiments, we want to show two issues in studying interaction between drugs and domains.
1. **First problem:** That when a drug interacts with a single-domain protein (with domain X), even if we correctly conclude that it interacts with domain X, It may not interact with another single domain protein that has domain X. This is easy to check using Data. For this, we need some negative interaction data and for that, we can go to affinity data.  
2. **Second Problem:** is about multi-domain proteins and that is when a drug is interacting with a multi-domain protein (with domains X and Y), we can’t confidently say if this drug interacts with X or Y or both or either or neither meaning several cases are possible:  
    - The drug interacts with protein because it interacts directly with X
    - The drug interacts with protein because it interacts directly with Y
    - The drug interacts with protein because X and Y are both present
    - The drug interacts with protein because either of X or Y are present
    - The drug interacts because X and Y are present and they are in certain configuration with respect to each other or other extrinsic properties of the protein besides existence of X and Y.
    - The drug interacts for a completely irrelevant reason to existence of X or Y. 
    
There might be some overlap between the problem-1 and problem-2. But conceptually, we can say that first problem arises when trying to go from a drug-domain interaction to drug-protein interaction and the second problem arises when we go in the reverse direction. We want to see if we can quantitatively assess how prevalent these problems are or at least illuminate them as much as possible.

# from positive interactions to negative
This means we infer drug-domain interactions from drug interactions of single domain proteins, and then find examples where the same domain occurs in other proteins but doesn't interact with same drugs (we have a negative interaction for it in our dataset). for this, the negative interactions are very important. common drug-target interaction databases only have positive interactions and they assume lack of a pair in the dataset to mean lack of interaction, which is obviously not correct. However there are some researches that also collect negative interaction data like [Coelho2016](https://doi.org/10.1371/journal.pcbi.1005219) where they have used affinity data to extract some negative interactions

# First attempt using Coelho2016 datset
This dataset is based on [Coelho2016](https://doi.org/10.1371/journal.pcbi.1005219) paper and contains negative and positive interactions. Negative interactions are extracted from BindingDB and BioLip databases, even though BioLip is questionable as a source of negative interactions because it is extracted from strucutres of drug-target complexes in the PDB, while we are more interested in those based on chemical assays.
To use this dataset to search for cases of problem-1, we create a table where for each pair of drug D and proteins P, where the protein is single-domain M, we list all other proteins Q that have the same domain M and divide them into three groups:
1. **Pos:** those that there is a positive interaction in the dataset between Q and D
1. **Neg:** those that there is a negative interaction in the dataset between Q and D
1. **Unk:** those that there is no interaction information in the dataset between Q and D

In [28]:

#first we read the dataset and 

interacts = dict() # for each pair in the dataset (key),  it shows the annotations True/False (interaction/non-interaction) if it exists in the dataset. we basically store all dataset infromation here.
uniprot_ids = set() # set of uniprot IDs for the purpose of collecting their pfam domain annotations
drugsof= dict() # we want positive interactions for single domain proteins so we store them here to be readily available

import pandas
for f in ["drugbank_DTIs_REAL_NEGS.txt","test_data_sc_and_bc.txt","yamanishi_DTIs_REAL_NEGS.txt"]:
    df = pandas.read_csv("DTIPred/"+f, sep = "\t", header = None)
    for index , row in df.iterrows():
        pid  = row[0]
        did  = row[1]
        interaction_exist  = row[2]
        uniprot_ids.add(pid)
#         if (pid,did) in interacts:
#             if (interacts[(pid,did)] != interaction_exist):
#                 print ("error repeat", (pid,did))
#         else:
        interacts[(pid,did)] = interaction_exist 
        if interaction_exist == 1:               
            if pid in drugsof:
                drugsof[pid].append(did)
            else:
                drugsof[pid] = [did]
        
with open ("DTIPred/uniprotids.txt", "w") as pf:
    pf.writelines("\n".join(uniprot_ids))
    
        
        

In [42]:

# here, we read the domain annotations we have downloaded from uniprot.

import pandas

def extract_items(field):
    if ";" not in field:
        return []
    else:
        spl = field.split(";")
        for s in spl:
            if len(s) <=1:
#                 print(s)
                spl.remove(s)
        return spl

proteinswith = dict() #for each domain (pfam ID), this will store the set of proteins (uniprot IDs) that have this domain
domainsof  = dict () #for each protein (uniprot ID), this will store the list of domains (pfam IDs) of that protein
df = pandas.read_csv("DTIPred/uniprotids_annnots.tab", sep = "\t", converters={i: str for i in range(100)})

for index , row in df.iterrows():
    domain_field = row ["Cross-reference (Pfam)"]
    pid = row["yourlist:M20210201A94466D2655679D1FD8953E075198DA83D46A3C"]    
    if True: #conditions for considering a protien such as being human protein or being reviewed
            domain_list = extract_items(domain_field)
            domainsof[pid]= domain_list 
            for dom  in domain_list:
                if dom in proteinswith:
                    proteinswith[dom].append(pid)
                else:
                    proteinswith[dom]= [pid]
        
    
num_domains = {x:len(domainsof[x]) for x in domainsof.keys()}
one_domain = [x for x in domainsof.keys() if len(domainsof[x])==1]

In [ ]:

# here we do the calculations, meaning we prepare the table consisting of pairs of single domain proteins (P) and interacting drugs (D) the number of proteins falling to each of the three groups and the ID of these proteins are stored in the next columns

drug_level_examples = "onedomain-protein,domain,interacting_drug,num_pos,num_neg,num_unk,pos,neg,unk\n"
protein_level_exmples  = ""


for p in one_domain:
    m = domainsof[p][0]
    Q_set = proteinswith[m].copy()
    if p in Q_set:
        Q_set.remove(p)        
    if p in drugsof:
        D_set = drugsof[p]
        for d in D_set:
            negs = []
            pos = []
            unk= []
            for q in Q_set:
                if (q,d) in interacts:
                    if interacts[(q,d)]==1:
                        pos.append(q)
                    else:
                        negs.append(q)
                else:
                    unk.append(q)
            row_str= ",".join ([p,m,d,str(len(pos)),str(len(negs)),str(len(unk)),";".join(pos), ";".join(negs), ";".join(unk)])+"\n"
            drug_level_examples+= row_str
            
with open("result_drug_level.csv","w") as outf:
    outf.writelines(drug_level_examples)
    

The result of this experiments showed that we couldn't find occurance of the problem-1 with this dataset. This can be due to small number of negative interactions that we have which can be due the the dataset being old. Therefore, we recollect the negative interactions from BindingDB to do this experiment again.

# Second attempt using BindingDB
we downloaded the BindingDB in tsv format. There were few issues here. First of all, for affinity, there are several measures here including Ki, Kd, IC50, and EC50. The literature that use affinity to obtain negative interactions don't clarfiy which of these measures they have used except one preprint that says they use Ki or IC50, even though based on a search that I did Kd is the most relevant measure for durg binding to proteins. 
Another problem is that some of the rows (interactions) in the bindingDB don't have a uniprot ID or have multiple chains. these cases altogether constitute less than 13% of interactions in the dataset. So for now, we ignore them because it makes the life much easier. 

In [6]:
import pandas 
import numpy


col_names = pandas.read_csv("data/BindingDB_All_2021m0.tsv/BindingDB_All.tsv", sep = "\t", nrows=0).columns
# types_dict = {"Ki (nM)": float,"Kd (nM)": float,"IC50 (nM)": float,"EC50 (nM)": float}
# types_dict.update({col: str for col in col_names if col not in types_dict})
types_dict={col: str for col in col_names}

df = pandas.read_csv("data/BindingDB_All_2021m0.tsv/BindingDB_All.tsv", sep = "\t",error_bad_lines=False,dtype=types_dict)
single_chain_mask = df["Number of Protein Chains in Target (>1 implies a multichain complex)"]=="1"
hasswissprot_mask = numpy.logical_not(df["UniProt (SwissProt) Primary ID of Target Chain"].isna())
haspubchemcid_mask = numpy.logical_not(df["PubChem CID"].isna())
singlechain_idcomplete_mask  = single_chain_mask & hasswissprot_mask & haspubchemcid_mask

b'Skipping line 896772: expected 193 fields, saw 205\nSkipping line 896798: expected 193 fields, saw 205\n'
b'Skipping line 929491: expected 193 fields, saw 205\n'
b'Skipping line 952161: expected 193 fields, saw 265\n'
b'Skipping line 1355895: expected 193 fields, saw 241\n'
b'Skipping line 1469800: expected 193 fields, saw 241\nSkipping line 1469801: expected 193 fields, saw 241\nSkipping line 1469802: expected 193 fields, saw 241\nSkipping line 1469803: expected 193 fields, saw 241\nSkipping line 1469804: expected 193 fields, saw 241\nSkipping line 1469805: expected 193 fields, saw 241\nSkipping line 1469806: expected 193 fields, saw 241\nSkipping line 1469807: expected 193 fields, saw 241\nSkipping line 1469808: expected 193 fields, saw 241\n'
b'Skipping line 1483073: expected 193 fields, saw 205\n'
b'Skipping line 1502296: expected 193 fields, saw 241\nSkipping line 1502297: expected 193 fields, saw 241\nSkipping line 1502298: expected 193 fields, saw 241\nSkipping line 1502299: e

## Selction of the measure and threshold for affinity
We use the same criterion used in other papers for defining interactions and non-interactions:
- noninteraction: Ki<1000nM or IC50<1000nM
- interaction: Ki>30000nM or IC50>30000nM

In [7]:
import numpy

interaction_threshold = 1000
noninteraction_threshold = 30000

def valid_number(x):
    try:
        if type(x) == str:
            x = x.replace(">","")
            x = x.replace("<","")
            x = x.replace(" ","")
        a  = float(x)
        if a == 0:
            return False
        return a
    except:
        return False

easy_df = df.loc[singlechain_idcomplete_mask,:].copy()

    
bin_interaction = numpy.array([""] * len (easy_df))

Ki_col= easy_df["Ki (nM)"].values 
Kd_col= easy_df["Ki (nM)"].values 
IC50_col= easy_df["IC50 (nM)"].values 
EC50_col= easy_df["EC50 (nM)"].values 

conflict_rows = []
incomplete_rows = []
for i in range(len(easy_df)):
    num_pdecisions = 0
    num_ndecisions = 0
    Ki = valid_number(Ki_col[i])
    Kd = valid_number(Kd_col[i])
    IC50 = valid_number(IC50_col[i])
    EC50 = valid_number(EC50_col[i])

        
    if Ki:
        if Ki < interaction_threshold:
            bin_interaction[i] = True
            num_pdecisions += 1
        elif Ki> noninteraction_threshold:
            bin_interaction[i] = False
            num_ndecisions += 1
#     if Kd:
#         if Kd < interaction_threshold:
#             bin_interaction[i] = True
#             num_pdecisions += 1
#         elif Kd > noninteraction_threshold:
#             bin_interaction[i] = False
#             num_ndecisions += 1
    if IC50:
        if IC50 < interaction_threshold:
            bin_interaction[i] = True
            num_pdecisions += 1
        elif IC50 > noninteraction_threshold:
            bin_interaction[i] = False
            num_ndecisions += 1
#     if EC50:
#         if EC50 < interaction_threshold:
#             bin_interaction[i] = True
#             num_pdecisions += 1
#         elif EC50> noninteraction_threshold:
#             bin_interaction[i] = False
#             num_ndecisions += 1
    if (num_pdecisions>0) and (num_ndecisions>0) :
        conflict_rows.append((i,num_pdecisions,num_ndecisions))
        bin_interaction[i] = ""
    if (num_ndecisions<1) and (num_pdecisions<1):
        incomplete_rows.append(i)

print("number of rows with undecided binary interaction:", len(incomplete_rows))
print("number of rows with conflicting binary interaction:", len(conflict_rows))        
easy_df["bin_interaction"] = bin_interaction 
easy_df_easy_bin = easy_df.loc[easy_df["bin_interaction"] != "" & ,["UniProt (SwissProt) Primary ID of Target Chain","PubChem CID","bin_interaction"]].copy()
easy_df_easy_bin["bin_interaction"].value_counts()

number of rows with undecided binary interaction: 671858
number of rows with conflicting binary interaction: 8


T    894796
F    152399
Name: bin_interaction, dtype: int64

Here, we dump the list of proteins (uniprot IDs) so that we can go and download their pfam domains from the uniprot.

In [8]:
target_uniprots  = easy_df_easy_bin["UniProt (SwissProt) Primary ID of Target Chain"].values
unique_target_uniprots  = set(target_uniprots)
# with open("unique_uniprots.txt", "w") as outf:
#     outf.writelines("\n".join(unique_target_uniprots))

Or if we already have them we ignore this part and just read them from the downloaded file

In [10]:
# we read the domain annotations we have downloaded from uniprot.

import pandas

def extract_items(field):
    if ";" not in field:
        return []
    else:
        spl = field.split(";")
        for s in spl:
            if len(s) <=1:
#                 print(s)
                spl.remove(s)
        return spl

    
    
proteinswith = dict() #for each domain (pfam ID), this will store the set of proteins (uniprot IDs) that have this domain
domainsof  = dict () #for each protein (uniprot ID), this will store the list of domains (pfam IDs) of that protein
df = pandas.read_csv("temp/unique_uniprots_domains.tab", sep = "\t", converters={i: str for i in range(100)})

for index , row in df.iterrows():
    domain_field = row ["Cross-reference (Pfam)"]
    pid = row["yourlist:M20210208A94466D2655679D1FD8953E075198DA843865FQ"]    
    if True: #conditions for considering a protien such as being human protein or being reviewed
            domain_list = extract_items(domain_field)
            domainsof[pid]= domain_list 
            for dom  in domain_list:
                if dom in proteinswith:
                    proteinswith[dom].append(pid)
                else:
                    proteinswith[dom]= [pid]
        
    
num_domains = {x:len(domainsof[x]) for x in domainsof.keys()}
one_domain = [x for x in domainsof.keys() if len(domainsof[x])==1]

In [12]:

# here we do the calculations, meaning we prepare the table consisting of pairs of single domain proteins (P) and interacting drugs (D) the number of proteins falling to each of the three groups and the ID of these proteins are stored in the next columns

drug_level_examples = "onedomain-protein,domain,interacting_drug,num_pos,num_neg,num_unk,pos,neg,unk\n"
protein_level_exmples  = ""



#first we read the dataset and 

interacts = dict() # for each pair in the dataset (key),  it shows the annotations True/False (interaction/non-interaction) if it exists in the dataset. we basically store all dataset infromation here.
uniprot_ids = set() # set of uniprot IDs for the purpose of collecting their pfam domain annotations
drugsof= dict() # we want positive interactions for single domain proteins so we store them here to be readily available
num_occurrance = dict()



import pandas



for index , row in easy_df_easy_bin.iterrows():
    pid  = row["UniProt (SwissProt) Primary ID of Target Chain"]
    did  = row["PubChem CID"]
    interaction_exist  = row["bin_interaction"]
    uniprot_ids.add(pid)
    if (pid,did) in interacts:
        num_occurrance[(pid,did)] +=1
    else:
        interacts[(pid,did)] = interaction_exist
        num_occurrance[(pid,did)] = 1
        if interaction_exist == "T":
            if (type(did)== str) and (type (pid)==str):
                if pid in drugsof:
                    drugsof[pid].add(did)
                else:
                    drugsof[pid] = set([did])
        
# with open ("DTIPred/uniprotids.txt", "w") as pf:
#     pf.writelines("\n".join(uniprot_ids))
    
top_repetitions = sorted(num_occurrance.items(), key=lambda x: x[1], reverse=True)




pdq_triples =[]

for p in one_domain:
    m = domainsof[p][0]
    Q_set = proteinswith[m].copy()
    if p in Q_set:
        Q_set.remove(p)        
    if p in drugsof:
        D_set = drugsof[p]
        for d in D_set:
            negs = []
            pos = []
            unk= []
            for q in Q_set:
                if type(q)== float:
                    print("error-q:", q)
                if (q,d) in interacts:
                    if interacts[(q,d)]=="T":
                        pos.append(q)
                    else:
                        pdq_triples.append((p,d,q)) 
                        negs.append(q)
                else:
                    unk.append(q)
            if (type(p) == float):
                print("error-p:", p)
            if (type(m) == float):
                print("error-m:", m)
            if (type(d) == float):
                print("error-d:", d)                
            row_str= ",".join ([p,m,d,str(len(pos)),str(len(negs)),str(len(unk)),";".join(pos), ";".join(negs), ";".join(unk)])+"\n"
            drug_level_examples += row_str
            
with open("problem1examples_10_30.csv","w") as outf:
    outf.writelines(drug_level_examples)
    

## Extra conditions
Since we have come up with a lot of examples, we want to apply different extra conditions to farther filter the examples:
There are few extrac conditions that comes to mind.
1. more strict condition for being single domain (not more than 20 consecutive unannotated resideus)
2. being human protein
3. having pH and/or temperature annotation (and if still we have many examples, we can say temperature should be 37)

in each case, we want to see how many examples will be left

### more strict condition for being single-domain
    

In [57]:
import xmltodict
from os import path

import xml.etree.ElementTree as ET
import requests

template = 'http://pfam.xfam.org/protein/P09482?output=xml'

domain_data_dir = "temp/domain_details/"

blank_length_threshold = 20


def pfam_record_dict(p):
    p_path = domain_data_dir + p + ".xml"
    if not path.exist(p_path):
        url = "http://pfam.xfam.org/protein/"+p+"?output=xml"
        req = requests.get(url)
        with open (domain_data_dir+prot+".xml","w") as outf:
            outf.writelines(req.text)
            
    with open(p_path) as pf:
        p_dict = xmltodict.parse(pf.read())
    return p_dict

def pfam_domains(p):
    p_dict = pfam_record_dict(p)
    p_domains = p_dict["pfam"]["entry"]["matches"]["match"]
    return p_domains

def specie_name(p):
    p_dict  = pfam_record_dict(p)
    return p_dict["pfam"]["entry"]["taxonomy"]["@species_name"]


def is_single_domain(p):
    p_dict = pfam_record_dict(p)
    p_seq  = p_dict["pfam"]["entry"]["sequence"]["#text"]
    p_len  = len (p_seq)
    on_domain =[False]*p_len
    p_domains = p_dict["pfam"]["entry"]["matches"]["match"]
    if type(p_domains) != list:
        p_domains =[p_domains]
    for dom in p_domains:
        acc = dom["@accession"]
        # type = dom["@type"]
        begin = int(dom["location"]["@start"])-1
        end = int(dom["location"]["@end"])-1
        on_domain[begin:end]  = [True] * ((end-begin)+1)
    streak = 0 
    max_streak = 0
    for i in range(p_len):
        if on_domain[i]:
            streak = 0            
        else:
            streak += 1
            max_streak  = max(streak, max_streak)
            
    if max_streak > blank_length_threshold:
        return False
    else:
        return True
    return on_domain


In [58]:
single_domain_pdq_triples = []
for (pp,dd,qq) in pdq_triples:
    if is_single_domain(pp):
        single_domain_pdq_triples.append((pp,dd,qq))

In [61]:
len(single_domain_pdq_triples)

1459

### both proteins being a human protein

In [59]:
single_domain_human_pdq_triples = []
for (pp,dd,qq) in single_domain_pdq_triples:
    if (specie_name(pp)=="Homo sapiens (Human)") and (specie_name(qq) == "Homo sapiens (Human)"):
        single_domain_human_pdq_triples.append((pp,dd,qq))


In [60]:
len(single_domain_human_pdq_triples)

923

## Createing report of found examples

since we learned that even with imposing these restirctions, we have close to one thoasand examples left, we generate the report based on these examples


In [69]:
import matplotlib.pyplot as plt
import xmltodict
import pandas
import numpy


class StringConverter(dict):
    def __contains__(self, item):
        return True
    def __getitem__(self, item):
        return str
    def get(self, default=None):
        return str



domain_data_dir = "temp/domain_details/"
shared_dom_col= (0,0.608,0.62,0.8)
other_dom_col= (0.6,0.6,0.6,0.8)


def visualize(p,q,shared_domain):
    fig = plt.figure(figsize=[8, 2])
    # text_ax = fig.add_subplot(211)
    ax = fig.add_subplot(111)
    # text_ax.spines["right"].set_visible(False)
    # text_ax.spines["left"].set_visible(False)
    # text_ax.spines["top"].set_visible(False)
    # text_ax.spines["bottom"].set_visible(False)
    # text_ax.get_yaxis().set_visible(False)
    # text_ax.get_xaxis().set_visible(False)

    p_path = domain_data_dir+p+".xml"
    with open(p_path) as pf:
        p_dict = xmltodict.parse(pf.read())
    p_seq  = p_dict["pfam"]["entry"]["sequence"]["#text"]
    p_len  = len (p_seq)
    ax.hlines(2, 0, p_len, linewidth=2, color="grey")
    p_domains = p_dict["pfam"]["entry"]["matches"]["match"]
    
    if type(p_domains) != list:
        p_domains =[p_domains]
    for dom in p_domains:
        acc = dom["@accession"]
        # type = dom["@type"]
        begin = int(dom["location"]["@start"])
        end = int(dom["location"]["@end"])
        #if pfam_A
        if acc==shared_domain:
            col = shared_dom_col
        else:
            col = other_dom_col
        ax.hlines(2, begin, end, linewidth=10, color=col)

    q_path = domain_data_dir + q + ".xml"
    with open(q_path) as qf:
        q_dict = xmltodict.parse(qf.read())
    q_seq = q_dict["pfam"]["entry"]["sequence"]["#text"]
    q_len = len(q_seq)
    ax.hlines(1, 0, q_len, linewidth=2, color="grey")
    q_domains = q_dict["pfam"]["entry"]["matches"]["match"]
    if type(q_domains) != list:
        q_domains =[q_domains]
    for dom in q_domains:
        acc = dom["@accession"]
        # type = dom["@type"]
        begin = int(dom["location"]["@start"])
        end = int(dom["location"]["@end"])
        # if pfam_A
        if acc == shared_domain:
            col = shared_dom_col
        else:
            col = other_dom_col
        ax.hlines(1, begin, end, linewidth=10, color=col)

    h_rng = float(max(p_len,q_len))
    h_margin = h_rng/10
    ax.set_xlim(-h_margin, h_rng+h_margin)
    ax.set_ylim(0.5, 2.5)
    # ax.get_yaxis().set_visible(False)
    p_label = "P ["+p+"]"
    q_label  = "Q ["+q+"]"
    ax.set_yticks([1,2])
    ax.set_yticklabels([q_label,p_label])
    fig_fname ="outputs/promissings/"+p+"_"+d+"_"+q+".svg"
    fig.suptitle("Domain: " + shared_domain)
    fig.savefig(fig_fname)
    plt.close(fig)






# col_names = pandas.read_csv("data/BindingDB_All_2021m0.tsv/BindingDB_All.tsv", sep = "\t", nrows=0).columns


# df = pandas.read_csv("data/BindingDB_All_2021m0.tsv/BindingDB_All.tsv", sep = "\t",error_bad_lines=False,converters=StringConverter())
useful_cols = ["Ki (nM)","Kd (nM)","IC50 (nM)","EC50 (nM)","kon (M-1-s-1)","koff (s-1)","pH","Temp (C)"]
mdfile = "# Potential examples for problem 1:\n"


for (p,d,q) in single_domain_human_pdq_triples:
    m = domainsof[p][0]
    visualize(p,q,m)
    mdfile += "## Interaction between domain "+m+" and drug "+d+"\n\n"
    p_rows_mask = (easy_df["UniProt (SwissProt) Primary ID of Target Chain"] == p) & (easy_df["PubChem CID"] == d)
    q_rows_mask = (easy_df["UniProt (SwissProt) Primary ID of Target Chain"] == q) & (easy_df["PubChem CID"] == d)

    p_rows = easy_df.loc[p_rows_mask, useful_cols].copy()
    q_rows = easy_df.loc[q_rows_mask, useful_cols].copy()


    mdfile += "Single-domain protein (P) interacting with the drug: " + p +"\n\n"
    mdfile += p_rows.to_markdown(index = False)+"\n\n"


    mdfile += "Another (Q) protein with the same domain: " + q +"\n\n"
    mdfile += q_rows.to_markdown(index = False)+"\n\n"

    mdfile += "![]("+p+"_"+d+"_"+q+".svg)\n\n"

with open("outputs/promissings/doc.md", "w") as outf:
    outf.writelines(mdfile)




# TRASH BELOW

In [48]:
len(single_domain_pdq_triples)

1459

In [ ]:
pdq_triples = []

In [34]:
pdq_triples[0]

('P58154', '9836977', 'Q05941')

In [54]:
single_domain_pdq_triples = []
species = []
for (pp,dd,qq) in pdq_triples:
    dic = valid_single_domain(pp)
    species.append(dic["pfam"]["entry"]["taxonomy"]["@species_name"])


In [27]:
a[1:3] = [True]* ((3-1)+1)

In [28]:
a

[False, True, True, True, False, False, False, False]

In [8]:

deletions = set()
for k in interacts:
    if (type(k[0])!=str or type(k[1]) !=str):
        deletions.add(k)
for i in deletions:
    del interacts[i]
len(interacts)

817123

In [7]:
# a piece of code for checking and example
# it takes a protein p and a protein q and a drug d and prints all rows with p and d and then all rows with q and d 
# the columns that we want are the 
p = "P58154"
q = "P09482"
d = "86418"
useful_cols = ["UniProt (SwissProt) Primary ID of Target Chain","PubChem CID","Ki (nM)","Kd (nM)","IC50 (nM)","EC50 (nM)","kon (M-1-s-1)","koff (s-1)","pH","Temp (C)"]

##first print information about p 
p_rows = (df["UniProt (SwissProt) Primary ID of Target Chain"]== p) & (df["PubChem CID"]== d)
first_row_number = numpy.nonzero(numpy.array(p_rows))[0]
p_seq = df.loc[first_row_number,"BindingDB Target Chain  Sequence"].values[0]


q_rows = (df["UniProt (SwissProt) Primary ID of Target Chain"]== q) & (df["PubChem CID"]== d)
first_row_number = numpy.nonzero(numpy.array(q_rows))[0]
q_seq = df.loc[first_row_number,"BindingDB Target Chain  Sequence"].values[0]

 


In [15]:
import xml.etree.ElementTree as ET
import requests
template = 'http://pfam.xfam.org/protein/P09482?output=xml'

for prot in unique_target_uniprots:
    print("                                       \r"+"Downloading "+prot, end  ="")
    url = "http://pfam.xfam.org/protein/"+prot+"?output=xml"
    req = requests.get(url)
    with open ("temp/domain_details/"+prot+".xml","w") as outf:
        outf.writelines(req.text)
    

In [123]:
import xml.etree.ElementTree as ET
tree = ET.fromstring(req.text)
# root.getchildren()
# for child in root.iter("https://pfam.xfam.org/}pfam"):
#     print(child.tag,  child.text)
dd = etree_to_dict(tree)

<ipython-input-122-164c03218a96>:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  d = {t.tag : map(etree_to_dict, t.getchildren())}


In [99]:
for node in root.findall(".//{https://pfam.xfam.org/}match"):
    print(node.tag, node.attrib)

{https://pfam.xfam.org/}match {'accession': 'PF02931', 'id': 'Neur_chan_LBD', 'type': 'Pfam-A'}
{https://pfam.xfam.org/}match {'accession': 'PF02932', 'id': 'Neur_chan_memb', 'type': 'Pfam-A'}


In [1]:
import pandas

def extract_items(field):
    if ";" not in field:
        return []
    else:
        spl = field.split(";")
        for s in spl:
            if len(s) <=1:
#                 print(s)
                spl.remove(s)
        return spl
                
    
            
    

df = pandas.read_csv("uniprot.tab", sep ="\t",dtype=str)

n_rows, n_cols  = df.shape

uniprot_ids = df["Entry"]
pfam_ids = df["Cross-reference (Pfam)"].astype(str).values
chembl_ids = df["Cross-reference (ChEMBL)"].astype(str).values

domain_count = dict()
drug_count = dict()


for i in range (n_rows):
    domain_list = extract_items(pfam_ids[i])
    drug_list = extract_items(chembl_ids[i])
    for dom in domain_list:
        if dom in domain_count:
            domain_count[dom] += 1
        else:
            domain_count[dom] = 1
    
    for drug in drug_list:
        if drug in drug_count:
            drug_count[drug] += 1
        else:
            drug_count[drug] = 1
            
    

### Mapping human proteins to pfam domains and DrugBank drugs
this allows us to study the prevalence of mullti-domain prorteins. Each multi-domain protein is a case where the problem two can happen. 


The above scripts allows us to count for each domain the number of proteins that have that domain
we will follow the rest of the job later but for now we focus on the approach where affinity data are used for considering true negative interactinos.

In [1]:
!ls

data  initial_experiments.ipynb  outputs  README.md  temp


952161## Studying the first problem
for this, we want to collect all human proteins and thier interactions, and look at all single-domain proteins and see the support and confidence for  association between domains and interactions. For a certain domain $\text{M}$, the support will be the the fraction of human proteins that have domain $M$ and the confidene for a drug $D$ will be:

$$ \frac{\text{Number of proteins that have $M$ and interact with $D$}}{\text{Number of proteins with $M$}} $$

### Using negative interactions to find examples of problem one

Here we use a dataset from the [paper](https://doi.org/10.1371/journal.pcbi.1005219). In this paper they have used BindingDB and BioLIP to build a dataset of both positive and negative interactions between proteins and drugs. An affinity threshold have been used to consider them negative.

After having negative interactions we need to find the cases where a drug interacts with a protein with only domain M but it doesn't interact with the many other proteins that have domain M 

to find these examples we devise an algorithm:

   
    for each domain M:
        find all proteins that only have M as their domain and put them in set P
        D = {}
        for each protein p in P:
            add to D all drugs that ineract with P
        //on paper, since domain M interacts with all drugs in D, then all proteins with M should also interact with all drugs in D
        for each drug d in D:
        if any of the proteins Q in NEG(d) has domain D then save them save (p, M, q) in the set of couter examples


so for this we need to have set of all domains and sort them based on frequency. 
for each domain we need all of the proteins that have it and for each protein 
            
            
        

